# Assignment 1 – Backprop

In [9]:
import re
import random
from collections import defaultdict
import itertools
from abc import ABC, abstractmethod
import math

In [10]:
#@title Select and Parse Math Problems [do not change]

#@markdown select math problem

math_problem = {"problem": "exp(x) + 2", "in_vars": {"x": 1.0}}
print(math_problem)

infix = [['exp', 'x'], '+', 2]
in_vars = {"x": 1.0}
print(infix, in_vars)

{'problem': 'exp(x) + 2', 'in_vars': {'x': 1.0}}
[['exp', 'x'], '+', 2] {'x': 1.0}


In [11]:
#@title ToDo1: Builder Class

class Builder():
    def __init__(self, infix, in_vars):
        """
        infix: infix notation of the math problem, e.g. [['exp', 'x'], '+', 2]
        in_vars: dict of input variables, e.g. {"x": 1.0}
        """
        self.infix = infix
        self.in_vars = in_vars
        
        # Create variable names for intermediate nodes
        self.available_vars = [f'v{i}' for i in range(100)]
        self.available_vars.reverse()
        
        # Build the computation graph
        self.graph = self.build_graph(infix, in_vars)
    
    def build_graph(self, infix, in_vars):
        """
        Builds a computation graph from infix notation.
        Returns a dict where each key is a variable name and value is a dict with:
        - 'value': computed value (None initially for intermediate nodes)
        - 'function': operation to apply (None for input variables)
        - 'operands': list of operands (variable names or constants)
        - 'backwards': gradient value (used in backpropagation)
        """
        graph = {}
        
        # Initialize input variables
        for var in in_vars.keys():
            graph[var] = {
                'value': in_vars[var],
                'function': None,
                'operands': None,
                'backwards': 0
            }
        
        # Process the infix expression recursively
        self._process_infix(infix, graph, level=0)
        
        return graph
    
    def _process_infix(self, infix, graph, level=0):
        """
        Recursively processes infix notation and builds the computation graph.
        Returns the variable name representing this sub-expression.
        """
        # Base case: if it's a single value (variable or constant)
        if not isinstance(infix, list):
            return infix
        
        # Recursively process nested lists (sub-expressions)
        for i, item in enumerate(infix):
            if isinstance(item, list):
                var_name = self._process_infix(item, graph, level + 1)
                infix[i] = var_name
        
        # Now infix contains only variables, constants, and operators
        # Determine the operation and operands
        if len(infix) == 2:
            # Unary operation: ['exp', 'x'] or ['sin', 'v1']
            op, operand = infix
            var_name = self.available_vars.pop()
            graph[var_name] = {
                'value': None,
                'function': op,
                'operands': [operand],
                'backwards': 0
            }
            return var_name
        
        elif len(infix) == 3:
            # Binary operation: ['x', '+', 2] or ['v1', '*', 'v2']
            operand1, op, operand2 = infix
            var_name = self.available_vars.pop()
            graph[var_name] = {
                'value': None,
                'function': op,
                'operands': [operand1, operand2],
                'backwards': 0
            }
            return var_name
        
        else:
            raise ValueError(f"Invalid infix expression: {infix}")


In [12]:
#@title ToDo2: Operations

class Operator(ABC):

    @abstractmethod
    def f(self, a, b = None) -> float:
        raise NotImplementedError()
        return f_res

    @abstractmethod
    def df(self, a, b = None) -> list:
        raise NotImplementedError()
        return [df_res]

class Exp(Operator):

    def f(self, a, b = None):
        return math.exp(a)

    def df(self, a, b = None):
        return [math.exp(a)]

class Log(Operator):
    ## natural logarithm

    def f(self, a, b = None):
        ## ToDO: implement
        return math.log(a)

    def df(self, a, b = None):
        ## ToDO: implement
        return [1/a]

class Mult(Operator):

    def f(self, a, b):
        return a * b

    def df(self, a, b=None):
        return [b, a]

class Div(Operator):

    def f(self, a, b):
        ## ToDO: implement
        return a/b

    def df(self, a, b):
        ## ToDO: implement
        return [1/b, -a/(b*b)]

class Add(Operator):

    def f(self, a, b):
        ## ToDO: implement
        return a+b

    def df(self, a, b = None):
        ## ToDO: implement
        return [1,1]

class Sub(Operator):

    def f(self, a, b = None):
        ## ToDO: implement
        return a-b

    def df(self, a, b = None):
        ## ToDO: implement
        return [1, -1]

class Pow(Operator):

    def f(self, a, b):
        return a**b

    def df(self, a, b):
        if a <= 0: ## work-around: treat as unary operation if -a^b
            return [b * (a ** (b - 1))]
        else:
            return [b * (a ** (b - 1)), (a ** b) * math.log(a)]

class Sin(Operator):

    def f(self, a, b=None):
        ## ToDO: implement
        return math.sin(a)

    def df(self, a, b=None):
        ## ToDO: implement
        return [math.cos(a)]

class Cos(Operator):

    def f(self, a, b=None):
        ## ToDO: implement
        return math.cos(a)

    def df(self, a, b=None):
        ## ToDO: implement
        return [-math.sin(a)]

In [13]:
#@title ToDo 3: Executing

class Executor():

    def __init__(self, graph: dict, in_vars: dict = {}):
        """
        graph: computation graph in a data structure of your choosing
        in_vars: dict of input variables, e.g. {"x": 2.0, "y": -1.0}
        """
        self.graph = graph
        self.in_vars = in_vars
        self.fn_map = {"log": Log(), "exp": Exp(), "+": Add(), "-": Sub(), "^": Pow(), "sin": Sin(), "cos": Cos(), "*": Mult(), "/": Div()}
        self.output = -1
        self.derivative = {}

    ## forward execution____________________________

    def forward(self):
        ## ToDO: implement and set self.output

        # Go through variables in topological order
        for key in self.graph.keys():

            # If the variable isn't yet evaluated, evaluate it based on parent nodes
            if self.graph[key]['value'] is None:

                # Each variable has parent nodes (operands), and the function to apply on the parent nodes (fun_to_use)
                fun_to_use = self.graph[key]['function']
                operands = self.graph[key]['operands'].copy()
                
                # Find the values of the parent nodes needed to compute the value of the current variable
                num_operands = len(operands)
                if num_operands == 1:
                    if operands[0] in self.graph.keys():
                        operands[0] = self.graph[operands[0]]['value']
                elif num_operands == 2:
                    if operands[0] in self.graph.keys():
                        operands[0] = self.graph[operands[0]]['value']
                    if operands[1] in self.graph.keys():
                        operands[1] = self.graph[operands[1]]['value']

                computed_val = self.fn_map[fun_to_use].f(*operands)

                self.graph[key]['value'] = computed_val
      
        self.output = self.graph[list(self.graph.keys())[-1]]['value']

    ## backward execution____________________________

    def backward(self):
        ## ToDO: implement and set self.derivative
        self.derivative = {}
        
        # Go through all nodes and set all  backwards values to 0
        for key in self.graph.keys():
          self.graph[key]['backwards'] = 0

        # 1. Go through nodes in reverse topological order (starting from the back). Last variable has backwards value = 1.
        # 2. For all children of the current node, find calculate the product of the corresponding gradient multiplied by the current node value
        # 3. Add the value to the children node backwards_value
        # (No need for recursion - just do it iteratively)

        output_flag = 0

        for key in reversed(list(self.graph.keys())):
            # Set the derivative of output wrt itself to 1 (backwards value)
            if not output_flag:
                self.graph[key]['backwards'] = 1
                output_flag = 1

            # Calculate the gradient of current node wrt its parent nodes
            fun_to_use = self.graph[key]['function']

            parent_values = []

            operands = self.graph[key]['operands']
            if operands is not None:
              for operand in operands:
                if type(operand) is str:
                  parent_values.append(self.graph[operand]['value'])
                elif operand is not None:
                  parent_values.append(operand)
            
            if fun_to_use is not None:
              gradient = self.fn_map[fun_to_use].df(*parent_values)

            # Do the rest of the variables
            if self.graph[key]['operands'] is not None:
                for i, parent in enumerate(self.graph[key]['operands']):
                  if (parent is not None) and (type(parent) is str):  
                    self.graph[parent]['backwards'] += self.graph[key]['backwards']*gradient[i]

        # Final output
        for input_var in self.in_vars:
          self.derivative[input_var] = self.graph[input_var]['backwards']


In [14]:
#@title Test Function for Debugging [do not change]

infix = [['exp', 'x'], '+', 2]
in_vars = {"x": 1.0}
g = Builder(infix, in_vars)
print(g.graph)
e = Executor(g.graph, in_vars)
e.forward()
e.backward()
print("Output:", e.output)
print("Derivative:", e.derivative)

{'x': {'value': 1.0, 'function': None, 'operands': None, 'backwards': 0}, 'v0': {'value': None, 'function': 'exp', 'operands': ['x'], 'backwards': 0}, 'v1': {'value': None, 'function': '+', 'operands': ['v0', 2], 'backwards': 0}}
Output: 4.718281828459045
Derivative: {'x': 2.718281828459045}


In [15]:
#@title Test Function for Grading [do not change]

# Test with another problem
math_problem2 = {"problem": "x * 2 + 1", "in_vars": {"x": 3.0}}
infix2 = [['x', '*', 2], '+', 1]
in_vars2 = {"x": 3.0}

g2 = Builder(infix2, in_vars2)
e2 = Executor(g2.graph, in_vars2)
e2.forward()
e2.backward()
print("Output:", e2.output)
print("Derivative:", e2.derivative)

Output: 7.0
Derivative: {'x': 2}


## Summary

This notebook implements a complete automatic differentiation system (backpropagation) with the following components:

### 1. Builder Class
- Converts infix notation expressions into a computation graph
- Creates intermediate variables for sub-expressions
- Builds a topologically ordered graph structure

### 2. Operator Classes
Implemented operators with forward (`f`) and backward (`df`) methods:
- **Unary**: `exp`, `log`, `sin`, `cos`
- **Binary**: `+`, `-`, `*`, `/`, `^` (power)

### 3. Executor Class
- **Forward pass**: Evaluates the expression by computing values in topological order
- **Backward pass**: Computes gradients using the chain rule in reverse topological order

### Test Results
1. **Test 1**: `exp(x) + 2` with `x=1.0`
   - Output: 4.718 (= e^1 + 2)
   - Derivative: 2.718 (= e^1)

2. **Test 2**: `x * 2 + 1` with `x=3.0`
   - Output: 7.0 (= 3 * 2 + 1)
   - Derivative: 2 (constant derivative with respect to x)

The implementation correctly handles both simple and nested expressions, computing both function values and their derivatives automatically.